# CNN基本结构

简单说下CNN的基本结构，这块主要参考了以下资料：

- [零基础理解卷积神经网络](https://zhuanlan.zhihu.com/p/32472241)
- [卷积神经网络简介](https://zhuanlan.zhihu.com/p/25249694)
- [卷积神经网络CNN学习](https://zhuanlan.zhihu.com/p/31919653)
- [禅与奶罩识别艺术](https://zhuanlan.zhihu.com/p/25774111)。
- [能否对卷积神经网络工作原理做一个直观的解释？](https://www.zhihu.com/question/39022858/answer/81026163)

首先是一些简单背景。2012年就在图像识别领域迅速崛起，应用最多的领域也是图像处理。假如一张jpg图片480\*480大小，那么它在计算机中的数组就是480\*480\*3，这张图片是狗的图片，计算机如何识别其特征呢？计算机可以通过识别更低层次的特征（曲线，直线）来进行图像识别。具体点，就是**用卷积层识别这些特征，并通过更多层卷积层结合在一起，就可以像人类一样识别出爪子和腿之类的高层次特征**，从而完成任务。这正是CNN所做的事情的大概脉络。

它与生物学有点关联－－在一个系统中，一些特定的组件发挥特定的作用（视觉皮层中的神经元寻找各自特定的特征）。这是CNN背后的基本原理，不同的卷积核寻找图像中不同的特征。

接着就是神经网络结构了。CNN的工作流程是这样的：你把一张图片传递给模型，经过一些**卷积层**，**非线性化（激活函数）**，**池化**，以及**全连层**，最后得到结果。就像我们之前所说的那样，输出可以是单独的一个类型，也可以是一组属于不同类型的概率。接下来简单记录下各个层的作用。

## 卷积层

先介绍下关于卷积的概念，这个概念在信号学中是基础概念，可以参考这里：[“卷积”其实没那么难以理解](https://zhuanlan.zhihu.com/p/41609577)，核心一句话摘抄一下：卷积将是**过去所有连续信号经过系统的响应之后得到的在观察那一刻的加权叠加**。

一般的卷积运算是先翻转一个向量，然后再做滑动乘积相加运算，而在神经网络中，是没有翻转这一步的，更多内容可以参考：

- [CNN 入门讲解：什么是卷积（Convolution）?](https://zhuanlan.zhihu.com/p/30994790)
- [谈谈离散卷积和卷积神经网络](https://liam.page/2017/07/27/convolutions-and-convolution-neural-network/)
- [卷积神经网络（CNN）之一维卷积、二维卷积、三维卷积详解](https://www.cnblogs.com/szxspark/p/8445406.html)
- [【模型解读】从2D卷积到3D卷积，都有什么不一样](https://zhuanlan.zhihu.com/p/55567098)。
- [vdumoulin/conv_arithmetic](https://github.com/vdumoulin/conv_arithmetic)

一般卷积公式：
$$\int _{-\infty}^{\infty} f(\tau)g(x-\tau)d\tau$$

CNN中卷积公式（不翻转）：
$$\int _{-\infty}^{\infty} f(\tau)g(x+\tau)d\tau$$

同前面说的一样，其物理意义大概可以理解为：系统**某一时刻的输出是由多个输入共同作用（叠加）的结果**。放在图像分析里，f(x) 可以理解为**原始像素点**(source pixel)，所有的**原始像素点叠加起来，就是原始图**了。g(x)可以称为**作用点**，**所有作用点合起来我们称为卷积核**（Convolution kernel）

卷积核上**所有作用点依次作用于原始像素点后**（即乘起来），**线性叠加的输出结果**，即是最终卷积的输出，也是我们想要的结果，我们称为destination pixel. 如下图所示：

![](img/v2-c9b00043ba326451979abda5417bfcdf_hd.jpg)

最左边呢就是我们原始输入图像了，中间呢是卷积层，-8就是卷积的结果。

为啥要用卷积？这就涉及几个图像方面的概念，比如边缘检测，边缘检测基本内容是**先检测边缘**，然后**把边缘叠加到原来的边缘上**，原本图像边缘的值如同被加强了一般，亮度没有变化，但是更加锐利。

一般是什么操作去检测边缘呢？先来看一，二阶微分。

对于一维函数f（x），其一阶微分的基本定义是差值：
$$\frac{\partial f}{\partial x} = f(x+1)-f(x)$$

二阶微分定义成差分：
$$\frac{\partial ^2 f}{\partial x^2} = f(x+1)+f(x-1)-2f(x)$$

看个例子，看边缘的灰度分布图以及将一二阶微分作用于边缘上：

![](img/v2-a894947c4a277a8a389eb04934cee5a1_hd.jpg)

可以看到，在边缘（也就是台阶处），二阶微分值非常大，其他地方值比较小或者接近0 。那我们就会得到一个结论，**微分算子的响应程度与图像在用算子操作的这一点的突变程度成正比**，这样，**图像微分增强边缘和其他突变**（如噪声），而**削弱灰度变化缓慢**的区域。也就是说，微分算子（尤其是二阶微分），对边缘图像非常敏感。

看完一维的，看下二维图像函数的拉普拉斯算子：
$$\nabla ^2 f=\frac{\partial ^2 f}{\partial x^2}+\frac{\partial ^2 f}{\partial y^2} = f(x+1,y)+f(x-1,y)-2f(x,y)+f(x,y+1)+f(x,y-1)-2f(x,y)$$

放到图上看，以x,y 为坐标轴中心点，来重新表达这个算子，就可以是：

![](img/v2-121b2aa6333fb0369459535cbb5d5b69_hd.jpg)

这个就和上面提到的卷积核(Convolutional kernel)有点像了啦。将原图像和拉普拉斯图像叠加在一起，就可以得到锐化后的结果了，这里就不再赘述了，基本上这个概念应该也蛮清楚了。

额外补充一点：同样提取某个特征，经过不同卷积核卷积后效果也不一样（这是个重点，为什么说重点，因为CNN里面卷积核的大小就是有讲究的）。

关于卷积更多的形式，可以参考：

- [入门：概览深度学习中的卷积结构](https://zhuanlan.zhihu.com/p/29715598)
- [一文了解各种卷积结构原理及优劣](https://zhuanlan.zhihu.com/p/28186857)
- [CNN中千奇百怪的卷积方式大汇总](https://zhuanlan.zhihu.com/p/29367273)

那么在CNN中，卷积在做什么？是怎么起到作用的呢？

从更高角度来说，每一个卷积核都可以被看做**特征识别器**。我所说的特征，是指直线、简单的颜色、曲线之类的东西。比如：

![](img/v2-72360d0b12c1d67d50ecc373a92bee44_hd.jpg)

假设我们要把下面这张图片分类。让我们把我们手头的这个卷积核放在图片的左上角。

![](img/v2-e329ce17f47f3c616d071fdd093694b9_hd.jpg)

基本上，如果输入图像中**有与卷积核代表的形状很相似的图形**，那么**所有乘积的和会很大**。如果我们移动了卷积核，遇到的都是不相似的部分，那么得到的值小多了。**卷积层的输出是一张激活图**。所以，在单卷积核卷积的简单情况下，假设卷积核是一个曲线识别器，那么所得的激活图会显示出哪些地方最有可能有曲线。但这只是**一个卷积核的情况**，只有一个找出向右弯曲的曲线的卷积核。可以**添加其他卷积核**，比如识别向左弯曲的曲线的。**卷积核越多，激活图的深度就越深**，我们得到的**关于输入图像的信息就越多**。

那么进一步看看卷积层到底是怎么作用的。

- [CNN入门讲解：卷积层是如何提取特征的？](https://zhuanlan.zhihu.com/p/31657315)
- [卷积神经网络CNN完全指南终极版（一）](https://zhuanlan.zhihu.com/p/27908027).

现在已经知道卷积可以提取特征，但是也不能随机找图像的pixels进行卷积吧，想想卷积输出的特征图（feature map）,除了特征值本身外，还包含相对位置信息，比如人脸检测，眼睛，鼻子，嘴巴都是从上到下排列的。那么提取出的相应的特征值也是按照这个**顺序**排列的。所以卷积的方式也希望按照正确的顺序来。因此实现卷积运算最后的方式就是**从左到右，每隔x列Pixel，向右移动一次卷积核**进行卷积(x可以自己定义)。当已经到最右，就**从上到下，每隔X行pixel,向下移动一次卷积核**，移动完成，再继续如上所述，从左到右进行。比如：

![](img/v2-0e86ac3e69a31e47477f658b76842c7c_hd.jpg)
![](img/v2-8d0c46394cac2f192e236c7cffff2559_hd.jpg)

就这样，我们先从左到右，再从上到下，直到所有pixels都被卷积核过了一遍，完成输入图片的第一层卷积层的特征提取。

x叫作**stride**,就是步长的意思，如果我们x = 2, 就是相当每隔两行或者两列进行卷积。

此外，分量的pixel 外面还围了一圈0，这是补0（**zero padding**），如下图所示。

![](img/v2-f22065e2b3de556f5ce3fae27ae8244c_hd.jpg)

因为添了一圈0，实际上什么信息也没有添，但是同样是stride x=1 的情况下，补0比原来没有添0 的情况下进行卷积，从左到右，从上到下都多赚了2次卷积，这样第一层卷积层输出的特征图（feature map）仍然为5x5，**和输入图片的大小一致**。这样有什么好处呢：

- 获得更多更细致的特征信息，上面那个例子我们就可以获得更多的图像边缘信息
- 可以控制卷积层输出的特征图的size，从而可以达到控制网络结构的作用，还是以上面的例子，如果没有做zero-padding以及第二层卷积层的卷积核仍然是3x3, 那么第二层卷积层输出的特征图就是1x1，CNN的特征提取就这么结束了。

暂时总结下，一个卷积核，作用到一个图像上，最后的形式如下所示：

![](img/v2-683c8d63e22eef01a271a08016006d17_hd.png)

即得到了一个feature map，feature map是每一个feature从原始图像中提取出来的“特征”。其中的值，**越接近为1表示对应位置和feature的匹配越完整**，越是接近-1，表示对应位置和feature的反面匹配越完整，而值接近0的表示对应位置没有任何匹配或者说没有什么关联。

一个feature作用于图片产生一张feature map，对上面的底图来说，如果用的是3个feature，因此最终产生3个 feature map。一定要注意是多个卷积核，生成多个feature map，因为很多博客都会用一个卷积核举例，所以很容易看着看着就只记得一个卷积核了。

下面看下实例（pytorch）

Pytorch中可以使用torch.nn.functional中的conv1d或2d、3d函数，也可以直接构造nn.Conv1d或2d、3d 类对象

In [5]:
import torch
from torch.nn import functional as F

In [7]:
inputs = torch.randn(33, 16, 30)
filters = torch.randn(20, 16, 5)
F.conv1d(inputs, filters).shape

torch.Size([33, 20, 26])

Conv类对象举个二维的例子：

In [21]:
con = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 5, bias = False)
print(con.weight.shape)

torch.Size([3, 3, 5, 5])


In [22]:
import numpy as np

In [25]:
inputs1 = torch.Tensor(np.arange(216).reshape(2,3,6,6))
con(inputs1).shape

torch.Size([2, 3, 2, 2])

F.conv1d中，input 的各维度分别是 ($\text{minibatch} , \text{in_channels} , iW$),iW意思是输入input的宽度，后面kW意思就是kernel卷积核的宽度；weight 各维度是 $(\text{out_channels} , \frac{\text{in_channels}}{\text{groups}} , kW)$

Conv2d作用下，所有的输入被卷积到所有的输出。如上有三个输入通道，如一幅彩色图像。同样，输出通道的数量也是三个。每个输出通道都会成为所有输入通道的函数（groups默认是1）。

可以看到输出是每个卷积核针对每个输入有一套输出，所以输出是batch每个元素有卷积核数目个输出，最后两个维度是卷积后的宽和高，示意图如下所示

![](img/group1-1.png)

上面的 groups 默认值是1（groups的介绍可以参考[这里](https://iksinc.online/2020/05/10/groups-parameter-of-the-convolution-layer/)），当要选择默认值1以外的groups值时，所选的值必须是一个整数，以便输入通道的数量和输出通道的数量都能被这个数整除。 一个非默认的groups值允许我们创建多个路径，每个路径只连接输入通道的一个子集到输出通道。举个例子，假设我们有8个通道从中间卷积层出来，你想把它们分组卷积，产生四个输出通道。在这种情况下，分组参数的非默认值为2和4是可能的。让我们看看在groups=4的情况下，通道的分组是如何进行的。 在这种情况下，卷积层是下面这样：

In [30]:
con2 = nn.Conv2d(in_channels = 8, out_channels = 4, groups=4, kernel_size = 5, bias = False)
print(con2.weight.shape)

torch.Size([4, 2, 5, 5])


In [32]:
inputs2 = torch.Tensor(np.full([3,8,6,6],1))
con2(inputs2).shape

torch.Size([3, 4, 2, 2])

计算过程示意图如下所示：

![](img/screen-shot-2020-05-09-at-3.07.54-pm.png)

也就是说两个channel一组卷积。

再补充一个一维的例子，更容易看到具体的数值：

这里设置了groups为3，因为输入的channels也是3，卷积核个数为3，所以就是每个channel对应一个卷积核，因为batch为2，也就是说[0, 1, 2, 3]和[12, 13, 14, 15]用[0, 1]卷积，[4, 5, 6, 7]和[16, 17, 18, 19]用[2, 3]卷，最后8-11和20-23用[4,5]卷

In [38]:
inputs3 = torch.Tensor(np.arange(24).reshape(2,3,4))
filters3 = torch.Tensor(np.arange(6).reshape(3, 1, 2))
F.conv1d(inputs3, filters3, groups=3)

tensor([[[  1.,   2.,   3.],
         [ 23.,  28.,  33.],
         [ 77.,  86.,  95.]],

        [[ 13.,  14.,  15.],
         [ 83.,  88.,  93.],
         [185., 194., 203.]]])

如果我们想要batch内每个元素对应一个卷积核要怎么办呢？

就是说，输入张量本来是[[[0, 1, 2, 3],[4, 5, 6, 7],[8, 9, 10, 11]],[[12, 13, 14, 15],[16, 17, 18, 19],[20, 21, 22, 23]]]，卷积核是[0,1]和[2,3]

我们想要[0,1]去卷[0, 1, 2, 3],[4, 5, 6, 7],[8, 9, 10, 11] 得到 [1, 2, 3],[5, 6, 7],[9, 10, 11]，[2,3]去卷[12, 13, 14, 15],[16, 17, 18, 19],[20, 21, 22, 23]得到[63, 68, 73],[83, 88, 93],[103, 108, 113]

那我们需要变换下输入变量的维度，把本来的batch变换到channel上去，让channel是2，batch是3，也就是输入张量是[[[0, 1, 2, 3],[12, 13, 14, 15]],[[4, 5, 6, 7],[16, 17, 18, 19]],[[8, 9, 10, 11], [20, 21, 22, 23]]]，卷积核是[[[0, 1]],[[2, 3]]]

用permute函数能帮助实现：

In [62]:
# 实际是2个batch，现在是2个channel
inputs4 = torch.Tensor(np.arange(24).reshape(2,3,4)).permute([1, 0, 2])
filters4 = torch.Tensor(np.arange(4).reshape(2, 1, 2))
conv4 = F.conv1d(inputs4, filters4, groups=2)
conv4

tensor([[[  1.,   2.,   3.],
         [ 63.,  68.,  73.]],

        [[  5.,   6.,   7.],
         [ 83.,  88.,  93.]],

        [[  9.,  10.,  11.],
         [103., 108., 113.]]])

In [67]:
conv4.shape

torch.Size([3, 2, 3])

In [63]:
real_conv4 = conv4.permute([1, 0, 2])
real_conv4

tensor([[[  1.,   2.,   3.],
         [  5.,   6.,   7.],
         [  9.,  10.,  11.]],

        [[ 63.,  68.,  73.],
         [ 83.,  88.,  93.],
         [103., 108., 113.]]])

In [68]:
real_conv4.shape

torch.Size([2, 3, 3])

## 非线性激活层

本小节同样参考了：[卷积神经网络CNN完全指南终极版（一）](https://zhuanlan.zhihu.com/p/27908027)。

卷积层对原图运算多个卷积产生一组线性激活响应，而非线性激活层是对**之前的结果进行一个非线性的激活响应**。

在神经网络中用到最多的非线性激活函数是Relu函数，它的公式定义如下：
$$f(x)=max(0,x)$$
即，保留大于等于0的值，其余所有小于0的数值直接改写为0。

为什么要这么做呢？上面说到，卷积后产生的特征图中的值，越靠近1表示与该特征越关联，越靠近-1表示越不关联，而我们进行特征提取时，为了使得数据更少，操作更方便，就**直接舍弃掉那些不相关联的数据**。

具体效果如下图所示：

![](img/v2-f9af9fde70d5d3b7db5562956c6cc213_hd.png)

## Pooling 池化层

本小节还参考了：[CNN入门讲解：什么是采样层（pooling）](https://zhuanlan.zhihu.com/p/32299939)。

采样层实际上就是一个特征选择的过程。比如我们用边缘滤波器去卷积输入图片，得到的特征值矩阵如下：

![](img/v2-c03717a0e283578ab302075c1c1fe029_hd.jpg)

那么池化是干什么呢？实际操作就是在四个方格里选最大的那个（这里特指是maxpooling，先以它说明）－－即 9。

这个矩阵就是特征图，这个数字的含义，可以理解为能代表这个特征的程度，比如上一层卷积层的卷积核或者说过滤器是边缘过滤器，9的意思就代表在这个区域，**这一块部位最符合边缘特征**。Maxpooling 就是在这个区域内选出最能代表边缘的值，也就是9，然后丢掉那些没多大用的信息。为什么这么做呢？

一方面，卷积操作后，我们得到了一张张有着不同值的feature map，尽管数据量比原图少了很多，但还是过于庞大（比较深度学习动不动就几十万张训练图片），因此池化操作就可以发挥作用了，它最大的目标就是减少数据量。

另外，还有可以避免overfitting等。

那么池化具体怎么操作呢？非常类似卷积层的卷积核。比如：

![](img/v2-16b276fe8c010af144383da29336c1e9_hd.jpg)

可以理解为卷积核每空两格做一次卷积，卷积核的大小是2x2， 但是卷积核的作用是取这个核里面最大的值（即特征最明显的值），而不是做卷积运算。

池化层还有什么性质？它可以一定程度提高**空间不变性**，比如说平移不变性，尺度不变性，形变不变性。为什么会有空间不变性呢？因为上一层卷积本身就是对图像一个区域一个区域去卷积，因此对于CNN来说，
重要是单独区域的特征，以及特征之间的相对位置（而不是绝对位置）。

Pooling 层说到底还是一个特征选择，信息过滤的过程，也就是说我们损失了一部分信息，这是一个和计算性能的一个妥协，随着运算速度的不断提高，我觉得这个妥协会越来越小。现在有些网络都开始少用或者不用pooling层了。

另外，还有average pooling，实际上就是把filter 里面的所以值求一个平均值。

特征提取的误差主要来自两个方面：

- 邻域大小受限；
- 卷积层权值参数误差。

目前认为average pooling 和max-pooling 的主要区别在于：

- average -pooling能减小第一种误差，更多的保留图像的背景信息
- max-pooling能减小第二种误差，更多的保留纹理信息

更多池化的细节及实例可以参考 本文件夹下 3.2-pooling-layer.ipynb 

## 全连接层

本节还参考了：[卷积神经网络CNN完全指南终极版（二）](https://zhuanlan.zhihu.com/p/28173972)。

在传统的CNN结构中，还会有其他层穿插在卷积层之间。总的来说，他们**提供了非线性化，保留了数据的维度，有助于提升网络的稳定度并且抑制过拟合**。一个经典的CNN结构是这样的：

![](img/v2-24a86448bce734cc5f6cc97a264644b4_hd.jpg)

先回头想想，卷积层的卷积核的目的是识别特征，他们识别像曲线和边这样的低层次特征。但可以想象，如果想预测一个图片的类别，必须让网络有能力**识别高层次的特征**，例如手、爪子或者耳朵。第一层的输入是原始图片，可第二层的输入只是第一层产生的激活图，激活图的每一层都表示了低层次特征的出现位置。如果用一些卷积核处理它，得到的会是表示高层次特征出现的激活图。这些特征的类型可能是半圆（曲线和边的组合）或者矩形（四条边的组合）。随着卷积层的增多，到最后，你可能会得到可以识别手写字迹、粉色物体等等的卷积核。

网络最后的画龙点睛之笔是全连层。全连接层要做的，就是对之前的所有操作进行一个总结，给我们一个最终的结果。这一层接受输入（来自卷积层，池化层或者激活函数都可以），并输出一个N维向量，其中，N是所有有可能的类别的总数。它最大的目的是**对特征图进行维度上的改变**，来得到每个分类类别对应的概率值。

全连接层的形式和前馈神经网络（feedforward neural network）的形式一样，或者称为多层感知机（multilayer perceptron，MLP）。

全连接层，顾名思义就是全部都连接起来，让我们把它与卷积层对比起来看。这么说来的话前面的卷积层肯定就不是全连接了，没错，**卷积层采用的是“局部连接”的思想**，回忆一下卷积层的操作，比如用一个3X3的图与原图进行连接操作，很明显原图中只有一个3X3的窗口能够与它连接起来。

![](img/v2-2ced3a81d5977a3bf70f9721a2fd725e_hd.png)

那除窗口之外的、未连接的部分怎么办呢？ 我们都知道，采用的是**将窗口滑动起来的方法后续进行连接**。这个方法的思想就是“**参数共享**” ，参数指的就是filter，**用滑动窗口的方式，将这个filter值共享给原图中的每一块区域连接进行卷积运算**。

**局部连接**与**参数共享**是卷积神经网络最重要的两个性质！

再看全连接，全连接一个重要函数：Softmax，分类中的一个重要函数，输出的是每个对应类别的概率值。更多内容可以参考：[小白都能看懂的softmax详解](https://blog.csdn.net/bitcarmanlee/article/details/82320853)或者[CNN入门讲解：我的Softmax和你的不太一样](https://zhuanlan.zhihu.com/p/41784404)

具体过程如下所示，

![](img/v2-ed75d8e64864d8b57a63d9a57859c819_hd.png)

例子是三个特征图，直接每个元素都连接到softmax上。相当于直接一个全局卷积核，每个值都是1，直接flatten展开，然后再输入softmax分类。这样做的好处是大大减少特征位置对分类带来的影响，形象解释可以参考：[CNN 入门讲解：什么是全连接层（Fully Connected Layer）?](https://zhuanlan.zhihu.com/p/33841176)。不过里面说两层全连接时为了非线性，这块存疑了，因为非线性并不是层数提供的，而是激活函数提供的，这块在stack overflow上查了下，可以参考这里：[Idea behind how many fully-connected layers should be use in a general CNN network](https://stackoverflow.com/questions/47007658/idea-behind-how-many-fully-connected-layers-should-be-use-in-a-general-cnn-netwo)，有一个回答是这么说的，前面的卷积层更像是一些特征提取的，真正的神经网络就是最后的全连接。至于为什么是两层而不是更多层，可能有这样一些原因，包括两层不会太大，更多层也可能没有太多提升，还有对于CNN，全连接也不是最核心的。

全连接的作用还可以参考这个：[全连接层的作用是什么？](https://www.zhihu.com/question/41037974/answer/150522307)